# 实验6: 神经网络的脆弱性

In [1]:
import torch
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append(r"..")
import utils

In [2]:
BATCH_SIZE = 8
NUM_WORKERS = 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Data

In [3]:
from utils import ISIC2018Dataset
from torchvision import transforms
from torch.utils.data import DataLoader

In [4]:
trans_test = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406],
    #                      std=[0.229, 0.224, 0.225])
])

test_data = ISIC2018Dataset('../../Robust-Skin-Lesion-Diagnosis/Data/2018/Test_GroundTruth.csv',
                            '../../Robust-Skin-Lesion-Diagnosis/Data/2018/ISIC2018_Task3_Training_Input', trans_test)
num_test = len(test_data)

test_loader = DataLoader(test_data, BATCH_SIZE, num_workers=NUM_WORKERS)

## Model

In [5]:
import torch.nn as nn

class Normalize(nn.Module):
    def __init__(self, mean, std) :
        super(Normalize, self).__init__()
        self.register_buffer('mean', torch.Tensor(mean))
        self.register_buffer('std', torch.Tensor(std))
        
    def forward(self, input):
        # Broadcasting
        mean = self.mean.reshape(1, 3, 1, 1)
        std = self.std.reshape(1, 3, 1, 1)
        return (input - mean) / std


class NormModel(nn.Module):
    """
    Add a normalization layer to a network, in order to help to restrict the adversarial examples to images
    """
    def __init__(self, model, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
        super().__init__()
        self.norm_layer = Normalize(mean, std)
        self.net = model

    def forward(self, x):
        x = self.norm_layer(x)
        x = self.net(x)
        return x

In [6]:
model = torch.load('1-1-1.pkl')
model = NormModel(model).to(device)

## Adversarial Attacks

In [7]:
from attacks import Normal, FGSM, PGD
from sklearn import metrics

In [8]:
def robust_eval(Attack, params):
    atk = Attack(model, **params)
    label = []
    pred = []
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        x = atk(x, y)
        z = model(x)
        _, yhat = torch.max(z.data, 1)
        label = np.concatenate((label, y.to('cpu')), axis=-1)
        pred = np.concatenate((pred, yhat.to('cpu')), axis=-1)
        
    report = metrics.classification_report(label, pred, target_names=['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC'], digits=3)
    print("{}\n{}".format(atk, report))


In [9]:
Attacks = [Normal, FGSM, PGD]
params = [{'eps': 8/255}, {'eps': 0.007}, {'eps': 8/255, 'alpha': 2/255, 'steps': 10, 'random_start': False}]

for Attack, param in zip(Attacks, params):
    robust_eval(Attack, param)

Normal(model_name=NormModel, eps=0.03137254901960784)
              precision    recall  f1-score   support

         MEL      0.465     0.509     0.486       283
          NV      0.929     0.741     0.825      1671
         BCC      0.774     0.205     0.324       117
       AKIEC      0.000     0.000     0.000        83
         BKL      0.327     0.855     0.473       289
          DF      0.348     0.296     0.320        27
        VASC      0.580     0.853     0.690        34

    accuracy                          0.675      2504
   macro avg      0.489     0.494     0.445      2504
weighted avg      0.758     0.675     0.688      2504

FGSM(model_name=NormModel, eps=0.007, loss=CrossEntropyLoss())
              precision    recall  f1-score   support

         MEL      0.046     0.053     0.049       283
          NV      0.717     0.353     0.473      1671
         BCC      0.055     0.068     0.061       117
       AKIEC      0.000     0.000     0.000        83
         BKL   

In [10]:
def clean_eval():
    label = []
    pred = []
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        z = model(x)
        _, yhat = torch.max(z.data, 1)
        label = np.concatenate((label, y.to('cpu')), axis=-1)
        pred = np.concatenate((pred, yhat.to('cpu')), axis=-1)
        
    report = metrics.classification_report(label, pred, target_names=['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC'], digits=3)
    print("{}\n{}".format('CLEAN', report))

In [11]:
clean_eval()

CLEAN
              precision    recall  f1-score   support

         MEL      0.743     0.562     0.640       283
          NV      0.903     0.959     0.930      1671
         BCC      0.694     0.735     0.714       117
       AKIEC      0.667     0.578     0.619        83
         BKL      0.780     0.689     0.732       289
          DF      0.680     0.630     0.654        27
        VASC      0.769     0.882     0.822        34

    accuracy                          0.855      2504
   macro avg      0.748     0.719     0.730      2504
weighted avg      0.849     0.855     0.850      2504

